此程序是从同花顺基金网站上抓取基金的基本数据并入库，包括基金类型、基金公司、名称、代码等。

In [19]:
import requests
import lxml.etree as ET
import json

response = requests.get('http://fund.ijijin.cn/data/Net/info/all_rate_desc_0_0_1_9999_0_0_0_jsonp_g.html')
temp = response.content.strip('g()')
data = json.loads(temp)

同花顺网站上的数据是通过上述网址得到一个基金列表，是g(json_str)的格式。其中包含的内容比较多，这里只取它的有用的几项。

In [9]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('postgresql://localhost/finance_db')
Session = sessionmaker()
session = Session(bind=engine)

from dataModels import *

In [17]:
def safe_get_company(company_name):
    company = session.query(FundCompany).filter(FundCompany.company_name==company_name).scalar()
    if company:
        return company
    db_com = FundCompany(company_name=company_name)
    session.add(db_com)
    session.commit()
    return db_com

def safe_get_type(type_name):
    fund_type = session.query(FundType).filter(FundType.type_name==type_name).scalar()
    if fund_type:
        return fund_type
    db_type = FundType(type_name=type_name)
    session.add(db_type)
    session.commit()
    return db_type

In [18]:
raw['f960022']
for code, data in raw.iteritems():
    fund_code = code.strip('f')
    fund_name = data['name']
    fund_type_name = data['typename']
    fund_company_name = data['orgname']
    fund_type = safe_get_type(fund_type_name)
    fund_company = safe_get_company(fund_company_name)
    
    fund = Fund(fund_code=fund_code,fund_name=fund_name,fund_company=fund_company,fund_type=fund_type)
    session.add(fund)
    session.commit()